In [5]:
import sys
sys.path.append("..")

from ai_temporalDifference   import *
from environment import *
from IPython.display import clear_output
import tqdm
from multiprocessing import Process, Manager
import multiprocessing
import os
from itertools import product

In [51]:
manager = Manager()
globalKnowlage = manager.dict()

env = Board([4,4])

In [53]:
def simulationArena(iterations , knowlageDict):
    jimi = Agent(2,globalKnowlage)
    sumi = Agent(1,globalKnowlage)
    epsilon = 0.3
    jimi.initialValue = sumi.initialValue = 1

    for ep in range(iterations):
        jimi.setEpsilon(epsilon)
        sumi.setEpsilon(epsilon)

        while True:
        
            action = jimi.deside(env.boardMap)
            if action == (-1,-1):
                print("ERRRRR")
            env.put(2, action)

            if env.howIsTheWinner() == 2:
                jimi.giveReward(1,True)
                sumi.giveReward(-1,True)
                # print('\nWinner is 2')
                jimi.score+=1
                break
            
            elif env.isBoardFull():
                jimi.giveReward(0,True)
                sumi.giveReward(0,True)
                break

    ########################################
            
            action = sumi.deside(env.boardMap)
            if action == (-1,-1):
                print("ERRRRR")
            env.put(1, action)

            if env.howIsTheWinner() == 1:
                jimi.giveReward(-1,True)
                sumi.giveReward(1,True)
                # print('\nWinner is 1')
                sumi.score+=1
                break

            elif env.isBoardFull():
                jimi.giveReward(0,True)
                sumi.giveReward(0,True)
                break

        jimi.deside(env.boardMap)
        sumi.deside(env.boardMap)
        jimi.reset()
        sumi.reset()
        env.reset()

        if ep % 2 == 0:
            jimi.syncKnowlage()
            sumi.syncKnowlage()

        epsilon = epsilon*0.9 if epsilon> 0.1 else 0.1

    return [jimi.score,sumi.score]
    

In [54]:
with multiprocessing.Pool(16) as pool:
    scores = np.array(pool.starmap(simulationArena,[(1000,globalKnowlage)]*16))

print(np.mean(scores,0),len(globalKnowlage))

[473.9375 313.8125] 25028


In [55]:
jimi = Agent(2,globalKnowlage)

jimi.setEpsilon(0.0)
jimi.reset()
env.reset()

for _ in range(5):
    running = True
    while running:
        r = 0


        if env.isBoardFull():
            break
        
        action = jimi.deside(env.boardMap)

        if action == (-1,-1):
            print('Errrrrr')
            print(env.boardMap)

        
        env.put(2, action)

############# printing
        print()
        f_hash = hashState(env.boardMap)
        if f_hash in jimi.afterStateValues.keys():
            value = jimi.afterStateValues[f_hash]
        else:
            value = 0

        print("current after state value: ", value)

############# printing

        if env.howIsTheWinner() == 2:
            r = 1
            running = False
            print('\nWinner is 2')
        elif not env.isBoardFull():

            env.humanTurn()

        if env.howIsTheWinner() == 1:
            r = -1
            running = False
            print('\nWinner is 1')

        jimi.giveReward(r)

    clear_output(wait=True)
    jimi.reset()
    env.reset()


current after state value:  2.8137565068862154
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 2 0]]

current after state value:  2.554109977844779
[[1 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [2 0 2 0]]

current after state value:  2.943658937992108
[[1 0 0 0]
 [0 0 0 0]
 [0 0 0 2]
 [2 1 2 0]]

current after state value:  0
[[1 1 2 0]
 [0 0 0 0]
 [0 0 0 2]
 [2 1 2 0]]

current after state value:  0
[[1 1 2 2]
 [0 1 0 0]
 [0 0 0 2]
 [2 1 2 0]]


Exception: 1,3 is taken before!

In [20]:
jimi.setEpsilon(0.1)
jimi.reset()
sumi.setEpsilon(0.1)
sumi.reset()
env.reset()

tq = tqdm.trange(2000)

for ep in tq:

    running = True
    

    while running:
        
        action = jimi.deside(env.boardMap)
        if action == (-1,-1):
            print("ERRRRR")
        env.put(2, action)

        if env.howIsTheWinner() == 2:
            jimi.giveReward(1,True)
            sumi.giveReward(-1,True)
            # print('\nWinner is 2')
            jimi.score+=1
            break
        
        elif env.isBoardFull():
            jimi.giveReward(0,True)
            sumi.giveReward(0,True)
            break

########################################
        
        action = sumi.deside(env.boardMap)
        if action == (-1,-1):
            print("ERRRRR")
        env.put(1, action)

        if env.howIsTheWinner() == 1:
            jimi.giveReward(-1,True)
            sumi.giveReward(1,True)
            # print('\nWinner is 1')
            sumi.score+=1
            break

        elif env.isBoardFull():
            jimi.giveReward(0,True)
            sumi.giveReward(0,True)
            break

        

    jimi.deside(env.boardMap)
    sumi.deside(env.boardMap)
    jimi.learn()
    sumi.learn()
    jimi.reset()
    sumi.reset()
    env.reset()

    if ep % 100 == 0:
        tq.set_description(str(jimi.score)+' vs '+str(sumi.score))
        jimi.score,sumi.score = 0,0
    

print("jimi: ",len(jimi.afterStateValues), 'score: ',jimi.score)
print("sumi: ",len(sumi.afterStateValues), 'score: ',sumi.score)
jimi.score,sumi.score = 0,0

0